This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/clutter.html).  I recommend having both windows open, side-by-side!

In [ ]:
import time
import open3d as o3d
import numpy as np
from pydrake.all import (PointCloud, Rgba, RigidTransform, RotationMatrix,
                         Sphere, StartMeshcat)

from manipulation import running_as_notebook
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.mustard_depth_camera_example import MustardExampleSystem

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

# Estimating normals (and local curvature)

TODO: Add the version from depth images (nearest pixels instead of nearest neighbors), and implement it in DepthImageToPointCloud.

In [ ]:
def normal_estimation():
    system = MustardExampleSystem()
    context = system.CreateDefaultContext()

    meshcat.Delete()
    meshcat.DeleteAddedControls()
    meshcat.SetProperty("/Background", "visible", False)

    point_cloud = system.GetOutputPort("camera0_point_cloud").Eval(context)
    cloud = point_cloud.Crop(lower_xyz=[-.3, -.3, -.3], upper_xyz=[.3, .3, .3])
    meshcat.SetObject("point_cloud", cloud)

    kdtree = o3d.geometry.KDTreeFlann(cloud.xyzs())

    neighbors= PointCloud(40)
    AddMeshcatTriad(meshcat, "least_squares_basis", length=0.03, radius=0.0005)

    meshcat.AddSlider("point", min=0, max=cloud.size()-1, step=1, value=4165)
    meshcat.AddButton("Stop Normal Estimation")
    print("Press the 'Stop Normal Estimation' button in Meshcat to continue")
    last_index = -1
    while meshcat.GetButtonClicks("Stop Normal Estimation") < 1:
        index = round(meshcat.GetSliderValue("point"))
        if index == last_index:
            time.sleep(.1)
            continue
        last_index = index

        query = cloud.xyz(index)
        meshcat.SetObject("query", Sphere(0.001), Rgba(0, 1, 0))
        meshcat.SetTransform("query", RigidTransform(query))
        (num, indices,
         distances) = kdtree.search_hybrid_vector_3d(query=query,
                                                     radius=0.1,
                                                     max_nn=neighbors.size())

        neighbors.resize(num)
        neighbors.mutable_xyzs()[:] = cloud.xyzs()[:, indices]

        meshcat.SetObject("neighbors", neighbors,
                          rgba=Rgba(0, 0, 1), point_size=0.001)

        neighbor_pts = neighbors.xyzs().T
        pstar = np.mean(neighbor_pts,axis=0)
        prel = neighbor_pts - pstar
        W = np.matmul(prel.T, prel)
        w, V = np.linalg.eigh(W)
        R = np.fliplr(V)
        # Handle improper rotations
        R = np.diag([1, 1, np.linalg.det(R)]) @ R
        # Flip normals
        if R[0,2] < 0:
            R = -R
        meshcat.SetTransform("least_squares_basis", RigidTransform(
            RotationMatrix(R), query))

        if not running_as_notebook:
            break

    meshcat.DeleteAddedControls()

normal_estimation()
